In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

import jupyterlab_dash
import dash
import dash_html_components as html

from pathlib import Path

import plotly.express as px

import plotly.graph_objs as go

from dash.dependencies import Input, Output

import pandas as pd

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

march30 = pd.read_csv(processed_dir/'03-30-2020.csv')
march31 = pd.read_csv(processed_dir/'03-31-2020.csv')

march30.loc[:, 'date'] = pd.to_datetime('03-30-2020')
march31.loc[:, 'date'] = pd.to_datetime('03-31-2020')

df = march30.append(march31)

In [ ]:
df.head()

In [ ]:
days = pd.date_range('03/01/2020', '04/02/2020', normalize=True)
days = days.strftime('%m-%d-%Y')

In [ ]:
processed_dir

In [ ]:
days[0]

In [ ]:
processed_dir/f'{day}.csv'

In [ ]:
base_dir = Path().cwd().parent
processed_dir = base_dir/'data'/'processed'/'daily_report'

df = pd.DataFrame()
for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)

In [ ]:
def get_date_slider_marks(daterange=None):
"""
    daterange pd.Series
"""
    daterange.unique()
    
    

In [ ]:
df['date']

In [ ]:
pd.to_datetime(day)

In [ ]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import re

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = pd.DataFrame()  

for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = pd.to_datetime(day)
    df = df.append(temp)


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(2020,3,1),
        max_date_allowed=dt(2020,4,2),
        initial_visible_month=dt(2020,3,15),
        date=str(dt(2020, 3, 15))
    ),
    html.Div(id='output-container-date-picker-single')
])


@app.callback(
    [Output('graph-with-slider', 'figure'),
     Output('output-container-date-picker-single', 'children')],
    [Input('my-date-picker-single', 'date')])
def update_output(date):

    df_map = df[df['date'] == pd.to_datetime(date)]
    trace = go.Choropleth(
        locations=df_map['country'],
        locationmode='country names',
        z=df_map['confirmed'],
        text=df_map['country'],
        autocolorscale=False,
        colorscale="YlOrRd",
    )
    
    string_prefix = 'You have selected: '
    if date is not None:
        date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
        date_string = date.strftime('%B %d, %Y')

    return (
        {
            'data': [trace], 
            'layout': {
                'title_text':'2014 Global GDP',
                'geo': {
                    'showframe': False,
                    'showcoastlines': True,
                    'projection_type': 'equirectangular'
                }
            }
        },
        string_prefix + date_string
    )

viewer = jupyterlab_dash.AppViewer()
viewer.show(app)

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

import plotly.express as px

# df = px.data.gapminder().query("year>=1980&year<=2007")

df = pd.DataFrame()

for day in days:
    temp = pd.read_csv(processed_dir/f'{day}.csv')
    temp.loc[:, 'date'] = day
    df = df.append(temp)

slider_marks = {str(n): str(d) for n, d in enumerate(df['date'].unique())}

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='day-slider',
        min=0,
        max=2,
        value=list(slider_marks.values())[0],
        marks=slider_marks,
        step=None
    )
])


@app.callback(Output('graph-with-slider', 'figure'), [Input('day-slider', 'value')])
def update_figure(selected_year):
    df_map = df[df['date'] == pd.to_datetime(selected_year)]
    trace = go.Choropleth(
        locations=df_map['country'],
        locationmode='country names',
        z=df_map['confirmed'],
        text=df_map['country'],
        autocolorscale=False,
        colorscale="YlOrRd",
    )

    return {
        'data': [trace],
    }

In [ ]:
viewer = jupyterlab_dash.AppViewer()
viewer.show(app)

In [ ]:
viewer = jupyterlab_dash.AppViewer()
viewer.show(app)